In [19]:
import pandas as pd
from selenium import webdriver
from dotenv import load_dotenv
from googlemaps import convert
import googlemaps
import os
import requests
import json
from coordenadas import pasando_a_string
from coordenadas import get_coordenadas
from coordenadas import rellenando_datasetfsq
import foursquare
from config import dic_categorias
from pymongo import MongoClient

In [16]:
load_dotenv()

True

In [20]:
tokg = os.getenv("tokg")
tokf3 = os.getenv ("tokf3")
client2= MongoClient ("localhost:27017")

In [21]:
client = foursquare.Foursquare( access_token= tokf3 )
gmaps = googlemaps.Client(key= tokg )
db = client2.get_database("ciudades")

## Lista inicial

Después de intentar con las ciudades españolas, me paso a las internacionales, que tienen datos en foursquare

driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [7]:
url="https://es.wikipedia.org/wiki/Anexo:Ciudades_por_PIB"
driver.get(url)

In [8]:
tablaciudades= driver.find_element_by_xpath ("/html/body/div[3]/div[3]/div[5]/div[1]/table/tbody")

In [9]:
listacitis=[]
for dato in tablaciudades.find_elements_by_tag_name("tr"):
    fila=[d for d in dato.find_elements_by_tag_name ("td")]
    if len (fila)>1 :
        dia= {
            "id": fila[0].text,
            "ciudad": fila[1].text,
            "pais": fila[2].text,
            "PIB": fila [3].text,
            "PIBpc": fila [4].text,
            "Continente": fila [5].text
             }
        listacitis.append (dia)

In [10]:
ciudades = pd.DataFrame(listacitis)

In [11]:
ciudades.drop (["PIB", "PIBpc"],axis=1, inplace=True)

In [12]:
ciudadesini = pd.DataFrame(listacitis[:30])

In [13]:
ciudadesini.drop (["PIB", "PIBpc"],axis=1, inplace=True)

In [14]:
ciudades.to_csv ("../Data/ciudades.csv")

In [15]:
ciudadesini.to_csv ("../Data/ciudadesini.csv")

## Agregando coordenadas

Las incluyo en varios formatos, porque luego según la api te las pide de una forma u otra.

ciudadesini = pd.read_csv ("../Data/ciudadesini.csv")

In [16]:
ciudadesini.head(1)

,id,ciudad,pais,Continente
0,1,Tokio,Japón,Asia


In [17]:
get_coordenadas (ciudadesini, "ciudad", "pais")

,id,ciudad,pais,Continente,Lon,Lat,coords,coords2
0,1,Tokio,Japón,Asia,139.650311,35.676192,"{'lat': 35.6761919, 'lng': 139.6503106}","(35.6761919, 139.6503106)"
1,2,Nueva York,Estados Unidos,América,-74.005973,40.712775,"{'lat': 40.7127753, 'lng': -74.0059728}","(40.7127753, -74.0059728)"
2,3,Los Ángeles,Estados Unidos,América,-118.243685,34.052234,"{'lat': 34.0522342, 'lng': -118.2436849}","(34.0522342, -118.2436849)"
3,4,París,Francia,Europa,2.352222,48.856614,"{'lat': 48.856614, 'lng': 2.3522219}","(48.856614, 2.3522219)"
4,5,Seúl,Corea del Sur,Asia,126.977969,37.566535,"{'lat': 37.566535, 'lng': 126.9779692}","(37.566535, 126.9779692)"
5,6,Londres,Reino Unido,Europa,-0.127758,51.507351,"{'lat': 51.5073509, 'lng': -0.1277583}","(51.5073509, -0.1277583)"
6,7,Osaka,Japón,Asia,135.502253,34.693725,"{'lat': 34.6937249, 'lng': 135.5022535}","(34.6937249, 135.5022535)"
7,8,Shanghái,China,Asia,121.473701,31.230416,"{'lat': 31.230416, 'lng': 121.473701}","(31.230416, 121.473701)"
8,9,Chicago,Estados Unidos,América,-87.629798,41.878114,"{'lat': 41.8781136, 'lng': -87.6297982}","(41.8781136, -87.6297982)"
9,10,Moscú,Rusia,Europa,37.617300,55.755826,"{'lat': 55.755826, 'lng': 37.6172999}","(55.755826, 37.6172999)"


In [18]:
ciudadesini.coords2 = ciudadesini.coords2.apply (pasando_a_string)

In [19]:
ciudadesini.to_csv ("../Data/ciudadesini.csv")

## Y a cargar datos

In [6]:
ciudadesini = pd.read_csv ("../Data/ciudadesini.csv")

In [9]:
# reducido= ciudadesini[:10]

In [10]:
# reducido

,Unnamed: 0,id,ciudad,pais,Continente,Lon,Lat,coords,coords2,nrestaurantes,...,nparques,lparques,nhospitales,lhospitales,ncines,lcines,nteatros,lteatros,nmuseos,lmuseos
0,0,1,Tokio,Japón,Asia,139.650311,35.676192,"{'lat': 35.6761919, 'lng': 139.6503106}","35.6761919, 139.6503",0,...,0,{},0,{},0,{},0,{},0,{}
1,1,2,Nueva York,Estados Unidos,América,-74.005973,40.712775,"{'lat': 40.7127753, 'lng': -74.0059728}","40.7127753, -74.0059",0,...,0,{},0,{},0,{},0,{},0,{}
2,2,3,Los Ángeles,Estados Unidos,América,-118.243685,34.052234,"{'lat': 34.0522342, 'lng': -118.2436849}","34.0522342, -118.243",0,...,0,{},0,{},0,{},0,{},0,{}
3,3,4,París,Francia,Europa,2.352222,48.856614,"{'lat': 48.856614, 'lng': 2.3522219}","48.856614, 2.3522219",0,...,0,{},0,{},0,{},0,{},0,{}
4,4,5,Seúl,Corea del Sur,Asia,126.977969,37.566535,"{'lat': 37.566535, 'lng': 126.9779692}","37.566535, 126.97796",0,...,0,{},0,{},0,{},0,{},0,{}
5,5,6,Londres,Reino Unido,Europa,-0.127758,51.507351,"{'lat': 51.5073509, 'lng': -0.1277583}","51.5073509, -0.12775",0,...,0,{},0,{},0,{},0,{},0,{}
6,6,7,Osaka,Japón,Asia,135.502253,34.693725,"{'lat': 34.6937249, 'lng': 135.5022535}","34.6937249, 135.5022",0,...,0,{},0,{},0,{},0,{},0,{}
7,7,8,Shanghái,China,Asia,121.473701,31.230416,"{'lat': 31.230416, 'lng': 121.473701}","31.230416, 121.47370",0,...,0,{},0,{},0,{},0,{},0,{}
8,8,9,Chicago,Estados Unidos,América,-87.629798,41.878114,"{'lat': 41.8781136, 'lng': -87.6297982}","41.8781136, -87.6297",0,...,0,{},0,{},0,{},0,{},0,{}
9,9,10,Moscú,Rusia,Europa,37.617300,55.755826,"{'lat': 55.755826, 'lng': 37.6172999}","55.755826, 37.617299",0,...,0,{},0,{},0,{},0,{},0,{}


In [8]:
for nombre, codigo in dic_categorias.items():
    rellenando_datasetfsq (ciudadesini, "coords2", codigo , 1000, "n" + nombre, "l"+nombre)  

RateLimitExceeded: Quota exceeded

Me dá como resultados todo ceros, las búsqeudas hay que hacerlas con texto, y el código de categoria usarlo solo para cosas muy concretas.  ( esto lo he cambiado 2 veces, primero tenia texto, pero no me cogia muchos resultados para las ciudades chinas y japonesas, por eso probé con el código y me quedé sin peticiones)

## trabajando con base de datos con ceros:

In [26]:
ciudadesini.drop (["Unnamed: 0", "id"], axis=1, inplace=True)

In [37]:
ciudadesini["ciudad"].replace({"Washington D.C": "Wahington D.C."})
#me daba error por el ".", y la lié

In [42]:
ciudadesini

,ciudad,pais,Continente,Lon,Lat,coords,coords2,nrestaurantes,lrestaurantes,nplayas,...,nparques,lparques,nhospitales,lhospitales,ncines,lcines,nteatros,lteatros,nmuseos,lmuseos
0,Washington D. C.,Japón,Asia,139.650311,35.676192,"{'lat': 35.6761919, 'lng': 139.6503106}","35.6761919, 139.6503",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
1,Washington D. C.,Estados Unidos,América,-74.005973,40.712775,"{'lat': 40.7127753, 'lng': -74.0059728}","40.7127753, -74.0059",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
2,Washington D. C.,Estados Unidos,América,-118.243685,34.052234,"{'lat': 34.0522342, 'lng': -118.2436849}","34.0522342, -118.243",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
3,Washington D. C.,Francia,Europa,2.352222,48.856614,"{'lat': 48.856614, 'lng': 2.3522219}","48.856614, 2.3522219",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
4,Washington D. C.,Corea del Sur,Asia,126.977969,37.566535,"{'lat': 37.566535, 'lng': 126.9779692}","37.566535, 126.97796",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
5,Washington D. C.,Reino Unido,Europa,-0.127758,51.507351,"{'lat': 51.5073509, 'lng': -0.1277583}","51.5073509, -0.12775",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
6,Washington D. C.,Japón,Asia,135.502253,34.693725,"{'lat': 34.6937249, 'lng': 135.5022535}","34.6937249, 135.5022",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
7,Washington D. C.,China,Asia,121.473701,31.230416,"{'lat': 31.230416, 'lng': 121.473701}","31.230416, 121.47370",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
8,Washington D. C.,Estados Unidos,América,-87.629798,41.878114,"{'lat': 41.8781136, 'lng': -87.6297982}","41.8781136, -87.6297",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
9,Washington D. C.,Rusia,Europa,37.617300,55.755826,"{'lat': 55.755826, 'lng': 37.6172999}","55.755826, 37.617299",0,{},0,...,0,{},0,{},0,{},0,{},0,{}


In [39]:
ciudades= pd.read_csv ("../Data/ciudades.csv")

In [43]:
nombreciudades=ciudades.ciudad[:30]

In [48]:
nombreciudades[13] = "Washington D. C"

In [49]:
ciudadesini["ciudad"]= nombreciudades

In [50]:
ciudadesini

,ciudad,pais,Continente,Lon,Lat,coords,coords2,nrestaurantes,lrestaurantes,nplayas,...,nparques,lparques,nhospitales,lhospitales,ncines,lcines,nteatros,lteatros,nmuseos,lmuseos
0,Tokio,Japón,Asia,139.650311,35.676192,"{'lat': 35.6761919, 'lng': 139.6503106}","35.6761919, 139.6503",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
1,Nueva York,Estados Unidos,América,-74.005973,40.712775,"{'lat': 40.7127753, 'lng': -74.0059728}","40.7127753, -74.0059",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
2,Los Ángeles,Estados Unidos,América,-118.243685,34.052234,"{'lat': 34.0522342, 'lng': -118.2436849}","34.0522342, -118.243",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
3,París,Francia,Europa,2.352222,48.856614,"{'lat': 48.856614, 'lng': 2.3522219}","48.856614, 2.3522219",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
4,Seúl,Corea del Sur,Asia,126.977969,37.566535,"{'lat': 37.566535, 'lng': 126.9779692}","37.566535, 126.97796",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
5,Londres,Reino Unido,Europa,-0.127758,51.507351,"{'lat': 51.5073509, 'lng': -0.1277583}","51.5073509, -0.12775",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
6,Osaka,Japón,Asia,135.502253,34.693725,"{'lat': 34.6937249, 'lng': 135.5022535}","34.6937249, 135.5022",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
7,Shanghái,China,Asia,121.473701,31.230416,"{'lat': 31.230416, 'lng': 121.473701}","31.230416, 121.47370",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
8,Chicago,Estados Unidos,América,-87.629798,41.878114,"{'lat': 41.8781136, 'lng': -87.6297982}","41.8781136, -87.6297",0,{},0,...,0,{},0,{},0,{},0,{},0,{}
9,Moscú,Rusia,Europa,37.617300,55.755826,"{'lat': 55.755826, 'lng': 37.6172999}","55.755826, 37.617299",0,{},0,...,0,{},0,{},0,{},0,{},0,{}


In [52]:
for i in ciudadesini.ciudad: #funcion
    db.create_collection(i)

In [55]:
ciudadesini.head(1)

,ciudad,pais,Continente,Lon,Lat,coords,coords2,nrestaurantes,lrestaurantes,nplayas,...,nparques,lparques,nhospitales,lhospitales,ncines,lcines,nteatros,lteatros,nmuseos,lmuseos
0,Tokio,Japón,Asia,139.650311,35.676192,"{'lat': 35.6761919, 'lng': 139.6503106}","35.6761919, 139.6503",0,{},0,...,0,{},0,{},0,{},0,{},0,{}


In [130]:
lista=[] # aquí intento crear una lista de diccionarios para luego meter en la base de datos
for i in ciudadesini:
    for n in range (0, len (ciudadesini[i])):
        ciudad={
              i: ciudadesini[i][n]
                }
    lista.append (ciudad)

In [131]:
lista #pero solo me sale yakarta, ultima fila, he probado varias cosas pero no me salia, asi que intento con json

[{'ciudad': 'Yakarta'},
 {'pais': ' Indonesia'},
 {'Continente': 'Asia'},
 {'Lon': 106.84559899999999},
 {'Lat': -6.2087634000000005},
 {'coords': "{'lat': -6.2087634, 'lng': 106.845599}"},
 {'coords2': '-6.2087634, 106.8455'},
 {'nrestaurantes': 0},
 {'lrestaurantes': {}},
 {'nplayas': 0},
 {'lplayas': {}},
 {'nocio_nocturno': 0},
 {'locio_nocturno': {}},
 {'nhoteles': 0},
 {'lhoteles': {}},
 {'nocio_deporte': 0},
 {'locio_deporte': {}},
 {'naire_libre': 0},
 {'laire_libre': {}},
 {'ncolegios': 0},
 {'lcolegios': {}},
 {'nguarderias': 0},
 {'lguarderias': {}},
 {'nuniversidades': 0},
 {'luniversidades': {}},
 {'naeropuertos': 0},
 {'laeropuertos': {}},
 {'nestaciones de tren': 0},
 {'lestaciones de tren': {}},
 {'nparques': 0},
 {'lparques': {}},
 {'nhospitales': 0},
 {'lhospitales': {}},
 {'ncines': 0},
 {'lcines': {}},
 {'nteatros': 0},
 {'lteatros': {}},
 {'nmuseos': 0},
 {'lmuseos': {}}]

In [125]:
jason=ciudadesini.to_json ()

In [ ]:
ciudadesini.to_json ("../Data/ci.json") # si cargo esto en la base de datos me crea una colección con:
ciudad: lista
loqsea:lista

In [126]:
jason

<bound method NDFrame.to_json of               ciudad             pais Continente         Lon        Lat  \
0              Tokio            Japón       Asia  139.650311  35.676192   
1         Nueva York   Estados Unidos    América  -74.005973  40.712775   
2        Los Ángeles   Estados Unidos    América -118.243685  34.052234   
3              París          Francia     Europa    2.352222  48.856614   
4               Seúl    Corea del Sur       Asia  126.977969  37.566535   
5            Londres      Reino Unido     Europa   -0.127758  51.507351   
6              Osaka            Japón       Asia  135.502253  34.693725   
7           Shanghái            China       Asia  121.473701  31.230416   
8            Chicago   Estados Unidos    América  -87.629798  41.878114   
9              Moscú            Rusia     Europa   37.617300  55.755826   
10             Pekín            China       Asia  116.407396  39.904200   
11           Colonia         Alemania     Europa    6.960279  50.93

In [135]:
jason2=ciudadesini.to_json ("../Data/ci2.json", orient="index") 
# si cargo esto en la base de datos, se crean documentos para cada ciudad. Pero por lo menos los datos salen bien.
# no me da tiempo a probar más

In [143]:
jason2=ciudadesini.to_json (orient="index")

In [157]:
ciudadesini.loc[1]

ciudad                                              Nueva York
pais                                            Estados Unidos
Continente                                             América
Lon                                                    -74.006
Lat                                                    40.7128
coords                 {'lat': 40.7127753, 'lng': -74.0059728}
coords2                                   40.7127753, -74.0059
nrestaurantes                                                0
lrestaurantes                                               {}
nplayas                                                      0
lplayas                                                     {}
nocio_nocturno                                               0
locio_nocturno                                              {}
nhoteles                                                     0
lhoteles                                                    {}
nocio_deporte                                          